In [301]:
from selenium.common import NoSuchElementException
from selenium import webdriver
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configuration de Selenium pour utiliser Chrome
driver = webdriver.Chrome()

# Charger la page Web
driver.get("https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page=5")
# Vérifier si le bouton "Reject All" existe
try:

    reject_all_button =driver.find_element(By.ID,"onetrust-reject-all-handler")
    # Si le bouton existe, cliquer dessus
    reject_all_button.click()
except NoSuchElementException:
    # Si le bouton n'existe pas, ne rien faire
    pass
for i in range(200):
    try:

        load_more_button = driver.find_element(By.XPATH, "//button[@data-discoverygridsmanager='btnLoadMore']")
        # Si le bouton existe, cliquer dessus
        load_more_button.click()
    except NoSuchElementException:
    # Si le bouton n'existe pas, ne rien faire
        print('The button does not exist')
        pass
    time.sleep(1) # Pause de 1 seconde

# Extraire le contenu HTML de la page Web mise à jour
html = driver.page_source

# Fermer la fenêtre du navigateur
driver.quit()

In [302]:
def get_info(soup):
    list = soup.find("ul", {"id" : "info"})
    film_details = {}
    title = soup.find("h1", {"data-qa" : "score-panel-movie-title"}).text

    film_details["title"] = title



    for child in list.children:
        if child.find("b") != -1 :
            category = child.find("b").text
            category = category.replace(':','')
            value = child.find("span").text
            value = value.strip().replace('\n', '').replace(" ",'').replace(",", ' ')
            film_details[category] = value
    return film_details

In [303]:
soup = BeautifulSoup(html)

In [304]:
film_divs= soup.findAll(class_='js-tile-link')

In [305]:
films_link = []
for film in film_divs:
    film_link = {}
    link_div = (film.find('a', {"data-track" : "scores"}))
    if link_div is not None:
        audiencescore = link_div.find("score-pairs")["audiencescore"]
        tomatoescore = link_div.find("score-pairs")["criticsscore"]

        film_link["link"] = link_div['href']
        film_link["audiencescore"] = audiencescore
        film_link["tomatoescore"]  = tomatoescore
        films_link.append(film_link)

In [306]:
base_url = 'https://www.rottentomatoes.com'
film_details_list = []
number_film = 0
for film_name in films_link:
    print(number_film, "/", len(films_link))

    url = base_url + film_name["link"]
    print(url)
    response = requests.get(url)
    if response.ok:
        soup = BeautifulSoup(response.content)
        infos = get_info(soup)
        infos["audiencescore"] = film_name["audiencescore"]
        infos["tomatoescore"] = film_name["tomatoescore"]
        film_details_list.append(infos)
    number_film += 1


0 / 1807
https://www.rottentomatoes.com/m/everything_everywhere_all_at_once
1 / 1807
https://www.rottentomatoes.com/m/cocaine_bear
2 / 1807
https://www.rottentomatoes.com/m/the_whale_2022
3 / 1807
https://www.rottentomatoes.com/m/boston_strangler_2023
4 / 1807
https://www.rottentomatoes.com/m/dragged_across_concrete
5 / 1807
https://www.rottentomatoes.com/m/luther_the_fallen_sun
6 / 1807
https://www.rottentomatoes.com/m/rrr
7 / 1807
https://www.rottentomatoes.com/m/shazam
8 / 1807
https://www.rottentomatoes.com/m/the_banshees_of_inisherin
9 / 1807
https://www.rottentomatoes.com/m/missing_2023
10 / 1807
https://www.rottentomatoes.com/m/a_man_called_otto
11 / 1807
https://www.rottentomatoes.com/m/john_wick
12 / 1807
https://www.rottentomatoes.com/m/puss_in_boots_the_last_wish
13 / 1807
https://www.rottentomatoes.com/m/triangle_of_sadness
14 / 1807
https://www.rottentomatoes.com/m/john_wick_chapter_3_parabellum
15 / 1807
https://www.rottentomatoes.com/m/all_quiet_on_the_western_front_2022

AttributeError: 'NoneType' object has no attribute 'text'

In [307]:
film_details_list[0]

{'title': 'Everything Everywhere All at Once',
 'Rating': 'R(SexualMaterial|Language|SomeViolence)',
 'Genre': 'Comedy Adventure Sci-fi Fantasy',
 'Original Language': 'English',
 'Director': 'DanKwan DanielScheinert',
 'Producer': 'JoeRusso AnthonyRusso MikeLarocca DanKwan DanielScheinert JonathanWang',
 'Writer': 'DanKwan DanielScheinert',
 'Release Date (Theaters)': 'Apr8 2022\xa0wide',
 'Release Date (Streaming)': 'Jun7 2022',
 'Box Office (Gross USA)': '$75.7M',
 'Runtime': '2h12m',
 'Distributor': 'A24',
 'Sound Mix': 'DolbyDigital',
 'Aspect Ratio': 'Flat(1.85:1)',
 'audiencescore': '86',
 'tomatoescore': '94'}

In [308]:
df = pd.DataFrame(data=film_details_list)
df = df.drop(["View the collection", "Aspect Ratio"], axis=1)

In [309]:
df.columns

Index(['title', 'Rating', 'Genre', 'Original Language', 'Director', 'Producer',
       'Writer', 'Release Date (Theaters)', 'Release Date (Streaming)',
       'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix',
       'audiencescore', 'tomatoescore'],
      dtype='object')

In [310]:
df = df[['title', 'tomatoescore','audiencescore','Rating', 'Genre', 'Original Language', 'Director', 'Producer',
       'Writer', 'Release Date (Theaters)', 'Release Date (Streaming)',
       'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix']]

In [314]:
df.to_csv("")

',title,tomatoescore,audiencescore,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Sound Mix\r\n0,Everything Everywhere All at Once,94,86,R(SexualMaterial|Language|SomeViolence),Comedy Adventure Sci-fi Fantasy,English,DanKwan DanielScheinert,JoeRusso AnthonyRusso MikeLarocca DanKwan DanielScheinert JonathanWang,DanKwan DanielScheinert,Apr8 2022\xa0wide,Jun7 2022,$75.7M,2h12m,A24,DolbyDigital\r\n1,Cocaine Bear,68,71,R(LanguageThroughout|DrugContent|BloodyViolenceandGore),Mystery&thriller Comedy,English,ElizabethBanks,PhilLord ChrisMiller ElizabethBanks MaxHandelman BrianDuffield AdityaSood MattReilly ChristineSun,JimmyWarden,Feb24 2023\xa0wide,Mar21 2023,$58.6M,1h35m,UniversalPictures,\r\n2,The Whale,64,91,R(SexualContent|Language|SomeDrugUse),Drama,English,DarrenAronofsky,,SamuelD.Hunter,Dec21 2022\xa0wide,Feb21 2023,$17.2M,1h57m,A24,DolbyDigital\r\n3,Boston Strangler,66,77,R(Language|So